In [ ]:
from bokeh.io import curdoc, show
from bokeh.models import ColumnDataSource, Select, TabPanel, Tabs, HoverTool
from bokeh.plotting import figure
from bokeh.layouts import column, row
import pandas as pd

In [ ]:
# Charger les données nettoyées
file_path = 'cleaned_indicateurs_macro_economiques.xlsx'
data_clean = pd.read_excel(file_path)


In [ ]:
# Préparer les années et les options pour les sélections
years = ['2018', '2019', '2020', '2021', '2022']
sectors = ['Secteur primaire', 'Secteur secondaire', 'Secteur tertiaire']

In [ ]:
# Fonction pour préparer les données pour Bokeh
def prepare_data(indicator_name):
    data = data_clean[data_clean['Indicateur'].str.contains(indicator_name)]
    return ColumnDataSource(data={
        'year': years,
        'value': data.iloc[0, 2:].astype(float)
    })

In [ ]:
# Fonction pour créer une figure avec hover
def create_figure(title, x_label, y_label, source, color, legend):
    p = figure(title=title, x_axis_label=x_label, y_axis_label=y_label, plot_height=300, plot_width=700)
    p.line(x='year', y='value', source=source, line_width=2, color=color, legend_label=legend)
    p.circle(x='year', y='value', source=source, size=8, fill_color="white", legend_label=legend)
    hover = HoverTool()
    hover.tooltips = [("Année", "@year"), ("Valeur", "@value")]
    p.add_tools(hover)
    return p

In [ ]:
# Fonction pour filtrer les données par secteur
def filter_data(sector):
    data = data_clean[data_clean['Indicateur'].str.contains(sector)]
    return ColumnDataSource(data={
        'indicator': data['Indicateur'],
        'value': data[years].mean(axis=1)
    })

In [ ]:
# Tableau de Bord 1 : PIB et Secteur Primaire
source_pib = prepare_data("PIB en volume")
p1_1 = create_figure("Évolution du PIB en volume", 'Année', 'Croissance (%)', source_pib, "blue", "Croissance du PIB")


In [ ]:
# Sélections dynamiques pour le secteur primaire
sector_select = Select(title="Sélectionner le secteur", value=sectors[0], options=sectors)
source_sector = filter_data(sectors[0])
p1_2 = figure(title="Valeurs ajoutées par secteur", x_axis_label='Indicateur', y_axis_label='Valeur', x_range=source_sector.data['indicator'], plot_height=300, plot_width=700)
p1_2.vbar(x='indicator', top='value', source=source_sector, width=0.9)
hover_sector = HoverTool()
hover_sector.tooltips = [("Indicateur", "@indicator"), ("Valeur", "@value")]
p1_2.add_tools(hover_sector)
p1_2.xaxis.major_label_orientation = 1.2


In [ ]:
def update_sector(attr, old, new):
    sector = sector_select.value
    new_data = filter_data(sector)
    source_sector.data = new_data.data
    p1_2.x_range.factors = new_data.data['indicator']

sector_select.on_change('value', update_sector)

dashboard1 = column(p1_1, sector_select, p1_2)


In [ ]:
# Tableau de Bord 2 : Secteur Secondaire
source_sec = prepare_data("Secteur secondaire")
p2_1 = create_figure("Valeurs ajoutées par secteur secondaire", 'Année', 'Croissance (%)', source_sec, "red", "Secteur secondaire")


In [ ]:
# Sélections dynamiques pour les sous-secteurs du secteur secondaire
sub_sectors = data_clean['Indicateur'][data_clean['Indicateur'].str.contains("Secteur secondaire")].unique()
sub_sector_select = Select(title="Sélectionner le sous-secteur", value=sub_sectors[0], options=list(sub_sectors))
source_sub_sector = prepare_data(sub_sectors[0])
p2_2 = create_figure("Valeurs ajoutées par sous-secteur", 'Année', 'Croissance (%)', source_sub_sector, "green", "Sous-secteur")
def update_sub_sector(attr, old, new):
    sub_sector = sub_sector_select.value
    new_data = prepare_data(sub_sector)
    source_sub_sector.data = new_data.data

sub_sector_select.on_change('value', update_sub_sector)

dashboard2 = column(p2_1, sub_sector_select, p2_2)

In [ ]:
# Tableau de Bord 3 : Secteur Tertiaire et Investissements
source_tert = prepare_data("Secteur tertiaire")
p3_1 = create_figure("Valeurs ajoutées par secteur tertiaire", 'Année', 'Croissance (%)', source_tert, "brown", "Secteur tertiaire")


In [ ]:
# Sélections dynamiques pour les investissements
investment_select = Select(title="Sélectionner l'indicateur d'investissement", value="Formation brute du capital fixe en volume", options=["Formation brute du capital fixe en volume"])
source_investment = prepare_data(investment_select.value)
p3_2 = create_figure("Investissements", 'Année', 'Évolution (%)', source_investment, "orange", "Investissement")

def update_investment(attr, old, new):
    investment = investment_select.value
    new_data = prepare_data(investment)
    source_investment.data = new_data.data

investment_select.on_change('value', update_investment)

dashboard3 = column(p3_1, investment_select, p3_2)


In [ ]:
# Combiner tous les tableaux de bord
tabs = Tabs(tabs=[
    TabPanel(child=dashboard1, title="PIB et Secteur Primaire"),
    TabPanel(child=dashboard2, title="Secteur Secondaire"),
    TabPanel(child=dashboard3, title="Secteur Tertiaire et Investissements")
])

# Ajouter les onglets au document courant
curdoc().add_root(tabs)